In [57]:
import requests
import json
import csv
from IPython.display import display
from IPython.display import Image
import networkx as nx
from py2cytoscape import util as cy

Function to create a cytoscape network from a csv file.

In [58]:
def network_from_csv(csv_filename, network_name):
    '''Creates a Cytoscape network from a .csv file'''
    
#Basic setup
    PORT_NUMBER = 1234
    BASE = 'http://localhost:' + str(PORT_NUMBER) + '/v1/'
    HEADERS = {'Content-Type': 'application/json'}

#Opening CSV file, creating lists from each row    
    with open(csv_filename, 'r') as f:
      reader = csv.reader(f, skipinitialspace=True, delimiter=',')
      your_list = list(reader)
    new_list=your_list[2:]

#Clearing session
    res = requests.delete(BASE + 'session')
    
#Creating netwok
    g = nx.MultiGraph(name=network_name)

    for lists in new_list:
        g.add_node(lists[2], node_1_locus_tag=lists[0], node_1_gene_name=lists[1], node_1_ortholog_group=lists[6], node_2_locus_tag=lists[3], node_2_gene_name=lists[4], node_2_ortholog_group=lists[7])

    for lists in new_list:
        g.add_edge(lists[2],lists[5], interaction_type=lists[10], source_data=lists[8], strain=lists[9] )

#Uncomment these if you want to see the number of nodes and edges in the notebook
    #print('Number of Nodes = ' + str(g.number_of_nodes()))
    #print('Number of Edges = ' + str(g.number_of_edges()))

    cytoscape_network = cy.from_networkx(g)

    res1 = requests.post(BASE + 'networks', data=json.dumps(cytoscape_network), headers=HEADERS)
    res1_dict = res1.json()
    new_suid = res1_dict['networkSUID']
    requests.get(BASE + 'apply/layouts/force-directed/' + str(new_suid))
    
#Uncomment this if you want to see the network in the notebook
    #return Image(BASE+'networks/' + str(new_suid) + '/views/first.png')

Same thing but with dictionaries, if the order of the columns is different but the header names are the same

In [62]:
def network_from_csv_dict(csv_filename, network_name):
    '''Creates a Cytoscape network from a .csv file through creating dictionaries'''
    
#Basic setup
    PORT_NUMBER = 1234
    BASE = 'http://localhost:' + str(PORT_NUMBER) + '/v1/'
    HEADERS = {'Content-Type': 'application/json'}

#Opening CSV file, creating dictionaries from each row    
    with open(csv_filename, 'r') as f:
      reader = csv.DictReader(f, skipinitialspace=True, delimiter=',')
      your_list = list(reader)

    new_list=your_list[1:]

#Clearing session
    res = requests.delete(BASE + 'session')
    
#Creating netwok
    g = nx.MultiDiGraph(name=network_name)

    for dicts in new_list:
        g.add_node(dicts['Node_1_Uniprot_ID'], node_1_locus_tag=dicts['Node_1_locus_tag'], node_1_gene_name=dicts['Node_1_gene_name_'], node_1_ortholog_group=dicts['Node_1_orthology_group_ID'], node_2_locus_tag=dicts['Node_2_locus_tag'], node_2_gene_name=dicts['Node_2_gene_name'], node_2_ortholog_group=dicts['Node_1_orthology_group_ID'])

    for dicts in new_list:
        g.add_edge(dicts['Node_1_Uniprot_ID'],dicts['Node_2_Uniprot_ID'], interaction_type=dicts['Layer'], source_data=dicts['Source'], strain=dicts['Strain'] )

#Uncomment these if you want to see the number of nodes and edges in the notebook
    #print('Number of Nodes = ' + str(g.number_of_nodes()))
    #print('Number of Edges = ' + str(g.number_of_edges()))

    cytoscape_network = cy.from_networkx(g)

    res1 = requests.post(BASE + 'networks', data=json.dumps(cytoscape_network), headers=HEADERS)
    res1_dict = res1.json()
    new_suid = res1_dict['networkSUID']
    requests.get(BASE + 'apply/layouts/force-directed/' + str(new_suid))
#Uncomment this if you want to see the network in the notebook
    #return Image(BASE+'networks/' + str(new_suid) + '/views/first.png')